In [3]:
!pip install labelme tensorflow opencv-python-headless albumentations matplotlib
#tensorflow-gpu opencv-python
!pip install matplotlib
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

import os
import time
import uuid
import cv2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


2023-10-30 14:58:14.108261: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-30 14:58:14.111337: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 14:58:14.153550: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 14:58:14.153598: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 14:58:14.153633: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')

print("GPUs:", gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.config.list_physical_devices('GPU')

GPUs: []


[]

In [7]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

#images = images.map(load_image)
#images.as_numpy_iterator().next()

train_img_dataset = tf.data.Dataset.list_files('/home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/train/images/*.jpg', shuffle=False)
train_img_dataset = train_img_dataset.map(load_image)
train_img_dataset = train_img_dataset.map(lambda x: tf.image.resize(x, (120,120)))
train_img_dataset = train_img_dataset.map(lambda x: x/255)

test_img_dataset = tf.data.Dataset.list_files('/home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/test/images/*.jpg', shuffle=False)
test_img_dataset = test_img_dataset.map(load_image)
test_img_dataset = test_img_dataset.map(lambda x: tf.image.resize(x, (120,120)))
test_img_dataset = test_img_dataset.map(lambda x: x/255)

eval_img_dataset = tf.data.Dataset.list_files('/home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/eval/images/*.jpg', shuffle=False)
eval_img_dataset = eval_img_dataset.map(load_image)
eval_img_dataset = eval_img_dataset.map(lambda x: tf.image.resize(x, (120,120)))
eval_img_dataset = eval_img_dataset.map(lambda x: x/255)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: /home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/train/images/*.jpg'

In [5]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']

train_labels_dataset = tf.data.Dataset.list_files('/home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/train/labels/*.json', shuffle=False)
train_labels_dataset = train_labels_dataset.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

test_labels_dataset = tf.data.Dataset.list_files('/home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/test/labels/*.json', shuffle=False)
test_labels_dataset = test_labels_dataset.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

eval_labels_dataset = tf.data.Dataset.list_files('/home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/eval/labels/*.json', shuffle=False)
eval_labels_dataset = eval_labels_dataset.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

eval_labels_dataset



InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: /home/nickage/git/computer-vision-cognition/notebooks/colab_images/aug_data/train/labels/*.json'

In [10]:
len(train_img_dataset), len(train_labels_dataset), len(test_img_dataset), len(test_labels_dataset), len(eval_img_dataset), len(eval_labels_dataset)

NameError: name 'train_img_dataset' is not defined

In [9]:
train_img_dataset.as_numpy_iterator().next()

NameError: name 'train_img_dataset' is not defined

In [ ]:
train = tf.data.Dataset.zip((train_img_dataset, train_labels_dataset))
train = train.shuffle(4080)
train = train.batch(8)
train = train.prefetch(4)

test = tf.data.Dataset.zip((test_img_dataset, test_labels_dataset))
test = train.shuffle(840)
test = train.batch(8)
test = train.prefetch(4)

eval = tf.data.Dataset.zip((eval_img_dataset, eval_labels_dataset))
eval = train.shuffle(1140)
eval = train.batch(8)
eval = train.prefetch(4)

In [ ]:
train.as_numpy_iterator().next()[1]

In [11]:
!pwd

/home/nickage/git/computer-vision-cognition/notebooks


In [7]:
from tensorflow.keras.models import load_model
import cv2
import tensorflow as tf
import numpy as np

facetracker = load_model('model/facetracker.h5')

cap = cv2.VideoCapture(1)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    #print("yhat", yhat)
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('FaceDetect', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 249ms/step


QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread (0xc1bc7e0).
Cannot move to target thread (0xc4d73d0)

QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread (0xc1bc7e0).
Cannot move to target thread (0xc4d73d0)

QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread (0xc1bc7e0).
Cannot move to target thread (0xc4d73d0)

QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread (0xc1bc7e0).
Cannot move to target thread (0xc4d73d0)

QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread (0xc1bc7e0).
Cannot move to target thread (0xc4d73d0)

QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread (0xc1bc7e0).
Cannot move to target thread (0xc4d73d0)

QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread (0xc1bc7e0).
Cannot move to target thread (0xc4d73d0)

QObject::moveToThread: Current thread (0xc4d73d0) is not the object's thread

1/1 [==============================] - 0s 90ms/step


KeyboardInterrupt: 

In [8]:
!pwd

/home/nickage/git/computer-vision-cognition/notebooks
